In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-2.4.6-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import to_date, when, datediff
import pyspark.sql.functions as psf

In [3]:
spark = SparkSession.builder.master("local").getOrCreate()

spark = SparkSession \
.builder \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
.getOrCreate()

In [4]:
# Read in the data here
clients_all = "E:\cmpoc\clients_all.csv"
clients_top = "E:\cmpoc\clients_top.csv"
investigations = "E:\cmpoc\investigations_20200601.csv"

schema_investigations = StructType([
    StructField("bus_dt", DateType()),
    StructField("prpc_lob_seq_id", IntegerType()),
    StructField("inqr_id", DoubleType()),
    StructField("client_id", DateType()),
    StructField("calc_rslv_dt", DateType()),
    StructField("case_entr_dt", DateType()),
    StructField("frst_rslv_dt", DateType()),
    StructField("last_ropned_dt", DateType()),
    StructField("ropn_cn", IntegerType()),
    StructField("inqr_amt", DoubleType()),
    StructField("inqr_amt_ccy", StringType()),
    StructField("case_own_nm", StringType())
])

In [5]:
df_clients_all = spark.read.csv(clients_all, header=True, inferSchema=True)
df_clients_top = spark.read.csv(clients_top, header=True, inferSchema=True)
df_investigations = spark.read.csv(investigations, header=True, inferSchema=True)

In [6]:
df_clients_all.head(5)

[Row(client_id=1000, client_name='Accenture'),
 Row(client_id=1001, client_name='Apple'),
 Row(client_id=1002, client_name='Google Corp'),
 Row(client_id=1003, client_name='Reliance'),
 Row(client_id=1004, client_name='Facebook Inc')]

In [7]:
df_clients_top.head(5)

[Row(client_id=1000, top_client_ind='Y'),
 Row(client_id=1001, top_client_ind='Y'),
 Row(client_id=1005, top_client_ind='Y'),
 Row(client_id=1006, top_client_ind='Y'),
 Row(client_id=1007, top_client_ind='Y')]

In [8]:
df_investigations.head(5)

[Row(bus_dt='01-06-2020', prpc_lob_seq_id=3, inqr_id=5000000, client_id=1000, calc_rslv_dt='01-06-2020', case_entr_dt='01-01-2020', frst_rslv_dt='05-01-2020', last_ropned_dt='05-02-2020', ropn_cn=1, inqr_amt=20000, inqr_amt_ccy='USD', case_own_nm='Stephen'),
 Row(bus_dt='01-06-2020', prpc_lob_seq_id=3, inqr_id=5000001, client_id=1000, calc_rslv_dt='01-06-2020', case_entr_dt='01-02-2020', frst_rslv_dt='28-02-2020', last_ropned_dt='28-03-2020', ropn_cn=1, inqr_amt=1, inqr_amt_ccy='USD', case_own_nm='Stephen'),
 Row(bus_dt='01-06-2020', prpc_lob_seq_id=3, inqr_id=5000002, client_id=1000, calc_rslv_dt='01-06-2020', case_entr_dt='01-03-2020', frst_rslv_dt='15-03-2020', last_ropned_dt='25-03-2020', ropn_cn=2, inqr_amt=12556, inqr_amt_ccy='USD', case_own_nm='Stephen'),
 Row(bus_dt='01-06-2020', prpc_lob_seq_id=3, inqr_id=5000003, client_id=1000, calc_rslv_dt='01-06-2020', case_entr_dt='01-04-2020', frst_rslv_dt='15-04-2020', last_ropned_dt='25-04-2020', ropn_cn=2, inqr_amt=77787, inqr_amt_ccy

In [9]:
df_investigations = df_investigations.withColumn('bus_dt',to_date(df_investigations.bus_dt, 'dd-MM-yyyy'))

In [10]:
df_investigations = df_investigations.withColumn('calc_rslv_dt',to_date(df_investigations.calc_rslv_dt, 'dd-MM-yyyy'))

In [11]:
df_investigations = df_investigations.withColumn('case_entr_dt',to_date(df_investigations.case_entr_dt, 'dd-MM-yyyy'))

In [12]:
df_investigations = df_investigations.withColumn('bus_dt',to_date(df_investigations.bus_dt, 'dd-MM-yyyy'))

In [13]:
df_investigations = df_investigations.withColumn('frst_rslv_dt',to_date(df_investigations.frst_rslv_dt, 'dd-MM-yyyy'))

In [14]:
df_investigations = df_investigations.withColumn('last_ropned_dt',to_date(df_investigations.last_ropned_dt, 'dd-MM-yyyy'))

In [15]:
df_investigations = df_investigations \
  .withColumn('year', df_investigations['bus_dt'].substr(1, 4)) \
  .withColumn('month', df_investigations['bus_dt'].substr(6, 2)) \
  .withColumn('day', df_investigations['bus_dt'].substr(9, 2))

In [16]:
df_investigations.head(5)

[Row(bus_dt=datetime.date(2020, 6, 1), prpc_lob_seq_id=3, inqr_id=5000000, client_id=1000, calc_rslv_dt=datetime.date(2020, 6, 1), case_entr_dt=datetime.date(2020, 1, 1), frst_rslv_dt=datetime.date(2020, 1, 5), last_ropned_dt=datetime.date(2020, 2, 5), ropn_cn=1, inqr_amt=20000, inqr_amt_ccy='USD', case_own_nm='Stephen', year='2020', month='06', day='01'),
 Row(bus_dt=datetime.date(2020, 6, 1), prpc_lob_seq_id=3, inqr_id=5000001, client_id=1000, calc_rslv_dt=datetime.date(2020, 6, 1), case_entr_dt=datetime.date(2020, 2, 1), frst_rslv_dt=datetime.date(2020, 2, 28), last_ropned_dt=datetime.date(2020, 3, 28), ropn_cn=1, inqr_amt=1, inqr_amt_ccy='USD', case_own_nm='Stephen', year='2020', month='06', day='01'),
 Row(bus_dt=datetime.date(2020, 6, 1), prpc_lob_seq_id=3, inqr_id=5000002, client_id=1000, calc_rslv_dt=datetime.date(2020, 6, 1), case_entr_dt=datetime.date(2020, 3, 1), frst_rslv_dt=datetime.date(2020, 3, 15), last_ropned_dt=datetime.date(2020, 3, 25), ropn_cn=2, inqr_amt=12556, in

In [17]:
df_investigations = df_investigations.withColumn('first_tat',datediff(df_investigations.frst_rslv_dt, df_investigations.case_entr_dt))

In [18]:
df_investigations = df_investigations.withColumn('last_tat',datediff(df_investigations.calc_rslv_dt, df_investigations.last_ropned_dt))

In [19]:
df_detail = df_investigations.join(df_clients_top, on=['client_id'], how='left_outer')

In [20]:
df_detail.head()

Row(client_id=1000, bus_dt=datetime.date(2020, 6, 1), prpc_lob_seq_id=3, inqr_id=5000000, calc_rslv_dt=datetime.date(2020, 6, 1), case_entr_dt=datetime.date(2020, 1, 1), frst_rslv_dt=datetime.date(2020, 1, 5), last_ropned_dt=datetime.date(2020, 2, 5), ropn_cn=1, inqr_amt=20000, inqr_amt_ccy='USD', case_own_nm='Stephen', year='2020', month='06', day='01', first_tat=4, last_tat=117, top_client_ind='Y')

In [21]:
df_detail.describe()

DataFrame[summary: string, client_id: string, prpc_lob_seq_id: string, inqr_id: string, ropn_cn: string, inqr_amt: string, inqr_amt_ccy: string, case_own_nm: string, year: string, month: string, day: string, first_tat: string, last_tat: string, top_client_ind: string]

In [22]:
df_summ = df_detail.groupBy('bus_dt','client_id').agg(\
                                                      psf.sum(df_detail.first_tat + df_detail.last_tat).alias('total_tat'),\
                                                      psf.avg(df_detail.first_tat + df_detail.last_tat).alias('avg_tat'),\
                                                      psf.sum('inqr_amt').alias('total_value'),\
                                                      psf.count('inqr_id').alias('rslv_cnt'))

In [23]:
df_summ = df_summ.withColumn('bus_dt',to_date(df_investigations.bus_dt, 'dd-MM-yyyy'))

In [24]:
df_summ = df_summ \
  .withColumn('year', df_summ['bus_dt'].substr(1, 4)) \
  .withColumn('month', df_summ['bus_dt'].substr(6, 2)) \
  .withColumn('day', df_summ['bus_dt'].substr(9, 2))

In [25]:
df_summ.head(10)

[Row(bus_dt=datetime.date(2020, 6, 1), client_id=1002, total_tat=-16, avg_tat=-4.0, total_value=1015930, rslv_cnt=4, year='2020', month='06', day='01'),
 Row(bus_dt=datetime.date(2020, 6, 1), client_id=1001, total_tat=61, avg_tat=20.333333333333332, total_value=901588, rslv_cnt=3, year='2020', month='06', day='01'),
 Row(bus_dt=datetime.date(2020, 6, 1), client_id=1005, total_tat=20, avg_tat=4.0, total_value=33825, rslv_cnt=5, year='2020', month='06', day='01'),
 Row(bus_dt=datetime.date(2020, 6, 1), client_id=1000, total_tat=407, avg_tat=81.4, total_value=119244, rslv_cnt=5, year='2020', month='06', day='01'),
 Row(bus_dt=datetime.date(2020, 6, 1), client_id=1003, total_tat=0, avg_tat=0.0, total_value=144000, rslv_cnt=2, year='2020', month='06', day='01')]

In [1]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

3.14172228
